# Combining Datasets: Concat and Append
_____________________

``Series`` and ``DataFrame``s are built for operations on data which

* come from different data sources

* involve anything **from** very straightforward **concatenation** of two different datasets, **to** more complicated **joins** and **merges** that correctly handle any overlaps between the datasets

* are fast and straightforward with using Pandas functions and methods  

In [8]:
import pandas as pd
import numpy as np

In [9]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

In [10]:
# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Class for displaying multiple ``DataFrame``s side by side with ``_repr_html_`` method, which IPython uses to display:

In [11]:
class display:
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

## 1. Simple Concatenation with ``pd.concat()``
------------------------

#### 1.1. Concatenation of ``Series`` objects
-----------------

* The combination of options of the ``pd.concat()`` allows a wide range of possible behaviors when **joining** two datasets

In [12]:
pd.concat?

Signature:
pd.concat(
    objs: 'Iterable[NDFrame] | Mapping[Hashable, NDFrame]',
    axis: 'Axis' = 0,
    join: 'str' = 'outer',
    ignore_index: 'bool' = False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity: 'bool' = False,
    sort: 'bool' = False,
    copy: 'bool' = True,
) -> 'DataFrame | Series'
Docstring:
Concatenate pandas objects along a particular axis with optional set logic
along the other axes.

Can also add a layer of hierarchical indexing on the concatenation axis,
which may be useful if the labels are the same (or overlapping) on
the passed axis number.

Parameters
----------
objs : a sequence or mapping of Series or DataFrame objects
    If a mapping is passed, the sorted keys will be used as the `keys`
    argument, unless it is passed, in which case the values will be
    selected (see below). Any None objects will be dropped silently unless
    they are all None in which case a ValueError will be raised.
axis : {0/'index', 1/'columns'}, defa

In [13]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [14]:
ser2.append(ser1)

C:\Users\user\AppData\Local\Temp\ipykernel_8072\2469681031.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ser2.append(ser1)


4    D
5    E
6    F
1    A
2    B
3    C
dtype: object

In [15]:
pd.concat([ser2, ser1])

4    D
5    E
6    F
1    A
2    B
3    C
dtype: object

#### 1.2. Concatenation of ``DataFrame`` objects
-------------

* By default, the concatenation takes place row-wise within the ``DataFrame`` (i.e., ``axis=0``)

In [16]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


* Specification of an axis along which concatenation will take place:

In [17]:
df3 = make_df('AB', [1, 2])
df4 = make_df('CD', [1, 2])
display('df3', 'df4', "pd.concat([df3, df4], axis=1)")

df3
    A   B
1  A1  B1
2  A2  B2

df4
    C   D
1  C1  D1
2  C2  D2

pd.concat([df3, df4], axis=1)
    A   B   C   D
1  A1  B1  C1  D1
2  A2  B2  C2  D2

In [18]:
display('df3', 'df4', "pd.concat([df3, df4])")

df3
    A   B
1  A1  B1
2  A2  B2

df4
    C   D
1  C1  D1
2  C2  D2

pd.concat([df3, df4])
     A    B    C    D
1   A1   B1  NaN  NaN
2   A2   B2  NaN  NaN
1  NaN  NaN   C1   D1
2  NaN  NaN   C2   D2

In [19]:
df5 = make_df('AB', [2, 3])
display('df1', 'df5', 'pd.concat([df1, df5], axis=1)')

df1
    A   B
1  A1  B1
2  A2  B2

df5
    A   B
2  A2  B2
3  A3  B3

pd.concat([df1, df5], axis=1)
     A    B    A    B
1   A1   B1  NaN  NaN
2   A2   B2   A2   B2
3  NaN  NaN   A3   B3

## 2. Duplicate indices and ways to handle it
_____________________________________________

#### 2.1. Preserving indices
-----------

* Pandas concatenation *preserves indices*, even if the result will have duplicate indices (important difference between ``np.concatenate`` and ``pd.concat`` ): 

In [20]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # make duplicate indices!
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


* ``verify_integrity=True`` -- catching the repeats as an error:  the concatenation will raise an exception if there are duplicate indices:

In [21]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### 2.2. Managing of duplication
-----------------
* ``ignore_index=True`` -- ignoring the index when itself does not matter: the concatenation will create a new integer index for the resulting ``Series``:

In [22]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


* Adding MultiIndex ``keys`` option to specify a label for the data sources; the result is a multiply indexed DataFrame:

In [23]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

In [24]:
dxy = pd.concat([x, y], keys=['x', 'y'])
dxy

A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

In [25]:
dxy.loc['x']

,A,B
0,A0,B0
1,A1,B1


#### 2.3. Concatenation in case of different sets of column names with joins 
--------------

* ``join='outer', join_axes=None`` -- default values -- outer joins --  the join is a union of the input columns, the entries for which no data is available are filled with NaN

In [26]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6], sort=False)' )

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], sort=False)
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

In [27]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

* ``join='inner'`` -- inner join -- using the common columns only  

In [28]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

## 3. Concatenation with ``Series.append()`` and ``DataFrame.append()``  
___________________

* ``Series`` and ``DataFrame`` objects have an ``append()`` method for direct concatenation 

* ``append()`` does not modify the original object – instead it creates a new object  (unlike ``append()`` and ``extend()``  of `` lists``)

* ``append()`` also **is not a very efficient**, because it involves creation of a new index *and* data buffer.

* In the case of multiple ``append`` operations, it is generally better to build a list of ``DataFrame``s and pass them all at once to the ``concat()`` function

In [ ]:
pd.Series.append?

In [ ]:
pd.DataFrame.append?

In [29]:
display('df1', 'df2', 'df1.append(df2)')

<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [30]:
display('x', 'y', 'x.append(y, ignore_index=True)')

<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<string>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [ ]:
display('x')